# Why bother with `edit`?

Without a proper framework or tools, loading data requires the reimplementation of base boiler plate code between projects. 

Rather than wasting Scientist's time on setting this up, EDIT aims to provide a unified interface to access any dataset the user has access to.

Currently EDIT uses `xarray` as it's base level data loader, so any `netcdf`, `zarr`, and other `xarray` supported structures are included by default.

## Data Loaders

### Location Specific 
`edit` is designed with the knowledge that each computing infrastructure hold it's data in a slightly different way.

Accordingly, an archive must be set up for each location to allow the data at said location to be loaded.

This example notebook, shall use the `NCI` (National Computing Infrastructure) specific archive, as this is the main development environment.

### Other Data 
In addition to these specific location data loaders, `edit` contains an `Intake`, and a `CDS` data loader, allowing data to be downloaded or accessed via a intake catalog.

Also, various `patterns` are implemented that allow to be saved out, and loaded as if it was an archive.

## Example

### ERA5

In [1]:
import edit.data

/g/data/eu56/hc7988/Projects/EDIT/data/src/edit/data/__init__.py:132: UserWarning: `edit` is under heavy development and may not continue to be supported.
  __python_warnings.warn("`edit` is under heavy development and may not continue to be supported.", UserWarning)


`edit` includes an auto import step, which seeks to identify the current computing platform and load the associated archive. Therefore by default, upon import of `edit.data`, the `NCI` specific archive is loaded.

In [4]:
print(f"NCI archive loaded: {hasattr(edit.data.archive, 'NCI')}")

NCI archive loaded: True


In [10]:
print(edit.data.archive.NCI.ERA5.__init__.__doc__)


        Setup ERA5 Indexer

        Args:
            variables (list[str] | str):
                Data variables to retrieve
            level (Literal[ERA5_LEVELS]):
                Model level of data, must be either "single", "pressure"
            resolution (Literal[ERA_RES], optional):
                Resolution of data, must be one of 'monthly-averaged','monthly-averaged-by-hour', 'reanalysis'. Defaults to 'reanalysis'.
            level_value: (int, optional):
                Level value to select if data contains levels. Defaults to None.
            transforms (Transform | TransformCollection, optional): 
                Base Transforms to apply.
                Defaults to TransformCollection().
        


In [15]:
ERA5_index = edit.data.archive.NCI.ERA5('t', level = 'pressure')
ERA5_index

ERA5
	Description                    ECWMF ReAnalysis v5
		 range                          1970-current
		 Documentation                  https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation


	Initialisation                 
		 level                          pressure
		 level_value                    None
		 product                        reanalysis
		 transforms                     {}
		 variables                      t
	Transforms                     
		 ConformNaming                  {'latitude': ['lat', 'Latitude', 'yt_ocean', 'yt'], 'longitude': ['lon', 'Longitude', 'xt_ocean', 'xt'], 'time': ['Time']}
		 Rename                         {'t2m': '2t', 'u10': '10u', 'v10': '10v'}

Now that we have an index with `t` specified as the variable to retrieve, the object can be indexed with a time.

In [19]:
ERA5_index['2023-01-01T00']

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, level: 37, time: 1)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * level      (level) int32 1 2 3 5 7 10 20 30 ... 850 875 900 925 950 975 1000
  * time       (time) datetime64[ns] 2023-01-01
Data variables:
    t          (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 6, 486, 1080), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-05-01 05:40:30 UTC+1000 by era5_replication_tools-1.12...
    license:      Licence to use Copernicus Products: https://apps.ecmwf.int/...
    summary:      ERA5 is the fifth generation ECMWF atmospheric reanalysis o...
    title:        ERA5 pressure-levels oper temperature 20230101-20230131

You can notice that in the `repr` for the index, a couple of `Transforms`, these are added by default, and can be specified by the `archive`. Their role is to 'transform' the data into a more usable and universal format.

So:
- Coordinates are renamed to a standard
- Some ERA5 variables are aligned to the variable name given in the path

### ACCESS-G
Now that we have data from ERA5, let's show how to get data from ACCESS-G

In [17]:
ACCESS_index = edit.data.archive.NCI.ACCESS.analysis('pl/air_temp', region = 'G')
ACCESS_index

ACCESS_Analysis
	Description                    Australian Community Climate and Earth-System Simulator
		 Documentation                  http://www.bom.gov.au/nwp/doc/access/NWPData.shtml


	Initialisation                 
		 data_interval                  (6, 'h')
		 datatype                       an
		 level_value                    None
		 region                         G
		 transforms                     {}
		 variables                      ['pl/air_temp']
	Transforms                     
		 ConformNaming                  {'latitude': ['lat', 'Latitude', 'yt_ocean', 'yt'], 'longitude': ['lon', 'Longitude', 'xt_ocean', 'xt'], 'time': ['Time']}
		 VariableTrim                   {'variables': ['air_temp']}

In [18]:
ACCESS_index['2023-01-01T00']

<xarray.Dataset>
Dimensions:    (time: 1, lvl: 27, latitude: 1536, longitude: 2048)
Coordinates:
  * time       (time) datetime64[ns] 2023-01-01
  * latitude   (latitude) float64 89.94 89.82 89.71 ... -89.71 -89.82 -89.94
  * longitude  (longitude) float64 0.08789 0.2637 0.4395 ... 359.6 359.7 359.9
  * lvl        (lvl) float32 1e+03 2e+03 3e+03 5e+03 ... 9.5e+04 9.75e+04 1e+05
Data variables:
    air_temp   (time, lvl, latitude, longitude) float32 dask.array<chunksize=(1, 15, 924, 1230), meta=np.ndarray>
Attributes:
    Conventions:   CF-1.5,ACDD-1.3
    institution:   Australian Bureau of Meteorology
    source:        APS3
    date_created:  20230101
    summary:       analysis data
    title:         analysis data
    base_date:     20230101
    base_time:     0
    modl_vrsn:     ACCESS-G
    expt_id:       0001

## Comparison
There is no meaningful difference between how the two indexes are used.

Therefore it is quite easy for a user to change their data source, and all the complex data finding is abstracted away.

## Patterns
A pattern index can be used to save and access data generated by the user,

There are a couple already implemented, and can be easily configured.

In [29]:
edit.data.patterns.__dir__()

['__name__',
 '__doc__',
 '__package__',
 '__loader__',
 '__spec__',
 '__path__',
 '__file__',
 '__cached__',
 '__builtins__',
 'utils',
 'default',
 'PatternIndex',
 'PatternTimeIndex',
 'PatternForecastIndex',
 'PatternVariableAware',
 'argument',
 'Argument',
 'ArgumentExpansion',
 'ArgumentExpansionVariable',
 'direct',
 'Direct',
 'TemporalDirect',
 'ForecastDirect',
 'DirectVariable',
 'ForecastDirectVariable',
 'TemporalDirectVariable',
 'expanded_date',
 'ExpandedDate',
 'TemporalExpandedDate',
 'ForecastExpandedDate',
 'ExpandedDateVariable',
 'ForecastExpandedDateVariable',
 'TemporalExpandedDateVariable',
 'static',
 'Static']

In [28]:
edit.data.patterns.ExpandedDate('temp', prefix = 'data_').search('2020-01-01T00')

PosixPath('/jobfs/108073067.gadi-pbs/tmp1cb8bhae/2020/01/01/data_20200101T0000.nc')

These can be setup to automatically split out variables from an `xarray` dataset, or allow temporal indexing